In [1]:
# Necessary imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import os
import neptune

In [2]:
# Variables needed for neptune
proxies = {
 "http": "http://172.16.2.30:8080",
 "https": "http://172.16.2.30:8080",
}

project_name='Hatespeech-CNERG/MH-Bert'
api_token='Api token hidden for security purposes'


In [3]:
# Initialise neptune for logging the experiment results
neptune.init(project_name,api_token=api_token,proxies=proxies)
neptune.set_project(project_name)

Project(Hatespeech-CNERG/MH-Bert)

In [4]:
# LASER embeddings for each data point are 1024 dimensional, with the ith value stored as ith column in the 
# csv file
cols = [str(i) for i in range(1024)]

In [5]:
# The languages on which we are running the baseline experiment.
langs = ['Arabic','English','German','Indonesian','Italian','Polish','Portugese','Spanish','French']

In [6]:
# Function to train and test the Logistic regression model on the entire training data.
# Parameters:
#    lang - target language on which we test the model
#   c - a hyperparameter for the logistic regression model. 
#       Refer to sklearn logistic regression docs (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
#       for explanation of this value
#   seed_val - for setting the random seed value for reproducibility.
def train_and_test_all(lang,c,seed_val):
    # Define the parameters of the experiment and create a neptune experiment with these parameters
    params = {}
    params['how_train'] = 'all'
    params['random_seed'] = seed_val
    params['lang'] = lang
    params['C'] = c
    name_one = lang+'_'+'all'+'_'+str(seed_val)
    neptune.create_experiment(name_one,params=params,send_hardware_metrics=False,run_monitoring_thread=False)
    neptune.append_tag('LR all')
    neptune.append_tag(lang)
    neptune.append_tag('only_hate')
    # Load the training dataset. All the language datasets are loaded in this case.
    train = pd.DataFrame()
    columns = cols+['label']
    for l in langs:
        df = pd.read_csv(os.path.join('../Dataset/embedded/train',l+'.csv'))
        df = df[columns]
        train = pd.concat([train,df],axis=0)
    # Shuffle the datasets
    train = train.sample(frac=1,random_state=seed_val).reset_index(drop=True)
    train_x = train[cols]
    train_y = train['label']
    # Initialise a logistic regression model. The class weights are taken as balanced to account for data imbalance
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed_val)
    # Train the model
    model.fit(train_x,train_y)
    # Read the validation and test sets
    val = pd.read_csv(os.path.join('../Dataset/embedded/val',lang+'.csv'))
    test = pd.read_csv(os.path.join('../Dataset/embedded/test',lang+'.csv'))
    test_x = test[cols]
    test_y = test['label']
    val_x = val[cols]
    val_y = val['label']
    print(lang,c,seed_val)
    print('Training complete!')
    # Get the predictions on the validation and test sets
    pred = model.predict(test_x)
    fscore = f1_score(test_y,pred,average='macro')
    accuracy = accuracy_score(test_y,pred)
    pred_val = model.predict(val_x)
    val_fscore = f1_score(val_y,pred_val,average='macro')
    val_accuracy = accuracy_score(val_y,pred_val)
    print("Macro F1: ",'{0:.6f}'.format(f1_score(test_y,pred,average='macro')))
    print('')
    # Log the results to neptune
    neptune.log_metric('val_fscore',val_fscore)
    neptune.log_metric('val_acc',val_accuracy)
    neptune.log_metric('test_fscore',fscore)
    neptune.log_metric('test_accuracy',accuracy)
    neptune.stop()
    return fscore

In [7]:
# Run the above function for all languages, and 4 chosen c values, for 5 seed values each.
for lang in langs:
    for c in [0.01,0.1,1,10]:
        for seed in [2018,2019,2020,2021,2022]:
            np.random.seed(seed)
            train_and_test_all(lang,c,seed)

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6768
Arabic 0.01 2018
Training complete!
Macro F1:  0.380052

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6771
Arabic 0.01 2019
Training complete!
Macro F1:  0.380052

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6774
Arabic 0.01 2020
Training complete!
Macro F1:  0.380052

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6775
Arabic 0.01 2021
Training complete!
Macro F1:  0.380052

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6778
Arabic 0.01 2022
Training complete!
Macro F1:  0.380052

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6779
Arabic 0.1 2018
Training complete!
Macro F1:  0.563219

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6782
Arabic 0.1 2019
Training complete!
Macro F1:  0.563219

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6784
Arabic 0.1 2020
Training complete!
Macro F1:  0.563219

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6786
Arabic 0.1 2021
Training 

/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 2018
Training complete!
Macro F1:  0.650261

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6794


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Arabic 1 2019
Training complete!
Macro F1:  0.650261

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-6796


...

French 10 2020
Training complete!
Macro F1:  0.445455

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-7102


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 2021
Training complete!
Macro F1:  0.445455

https://ui.neptune.ai/Hatespeech-CNERG/MH-Bert/e/SAN-7103


/home/saketh/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


French 10 2022
Training complete!
Macro F1:  0.445455

